In [32]:
import numpy as np
import scipy.io as sio
import h5py
import os
import napari
from qtpy.QtWidgets import QSlider, QVBoxLayout, QWidget, QLabel
from qtpy.QtCore import Qt

from DataViewer import DataViewer

In [33]:
# Load names
scan_name = 'Scan_4'
peak_pre = 'Peaks_'
vol_pre = 'Volume_'
ext = '.mat'
folder = './DataOutput/'

peak_name = folder + peak_pre + scan_name + ext
vol_name = vol_pre+scan_name+'.h5'

In [34]:
print(str(vol_name))
first_dir = os.getcwd()

peak_dict = sio.loadmat(peak_name)

os.chdir('DataOutput')
with h5py.File(vol_name, "r") as f:
    print(f'Keys: {f.keys()}')
    group = f['dataFinal'][:]
os.chdir(first_dir)


Volume_Scan_4.h5
Keys: <KeysViewHDF5 ['dataFinal']>


In [37]:
# Refine data
unordered_peaks = np.array(peak_dict.get('peaks'))
unordered_vol = np.array(group)

perm = [2,0,1]
peaks = unordered_peaks[:,perm]
volume = np.transpose(unordered_vol, tuple(perm))

print(unordered_peaks[0], peaks[0])
print(np.shape(unordered_vol), np.shape(volume))

[860 300 822] [822 860 300]
(852, 892, 922) (922, 852, 892)


In [36]:
viewer = napari.Viewer(ndisplay=3)
image_layer = viewer.add_image(volume, name='Volume', colormap='gray') # Plots in [z,y,x]
points_layer = viewer.add_points(peaks, size=10, face_color='red', name='Centers')

dock_widget = QWidget()
layout = QVBoxLayout()

label_min = QLabel("Min Z")
slider_min = QSlider(Qt.Horizontal)
slider_min.setMinimum(0)
slider_min.setMaximum(volume.shape[0]-1) # Uses z coord of data -1 to find max
slider_min.setValue(0)

label_max = QLabel("Max Z")
slider_max = QSlider(Qt.Horizontal)
slider_max.setMinimum(0)
slider_max.setMaximum(volume.shape[0]-1)
slider_max.setValue(volume.shape[0]-1)

layout.addWidget(label_min)
layout.addWidget(slider_min)
layout.addWidget(label_max)
layout.addWidget(slider_max)
dock_widget.setLayout(layout)
viewer.window.add_dock_widget(dock_widget, area='bottom')

def update_view():
    
    zmin = slider_min.value()
    zmax = slider_max.value()

    # Clip points to slab
    filtered_points = peaks[(peaks[:, 0] >= zmin) & (peaks[:, 0] <= zmax)]
    points_layer.data = filtered_points  # z, y, x order

    # Clip volume to slab
    filtered_data = volume[zmin:zmax+1, :, :]
    image_layer.data = np.pad(filtered_data, pad_width=[[zmin, volume.shape[0] - zmax - 1], [0, 0], [0, 0]], mode='constant', constant_values=0)

slider_min.valueChanged.connect(update_view)
slider_max.valueChanged.connect(update_view)
update_view()  # initialize

napari.run()